In [1]:
import os
import cv2
import math
import time
import json
import pandas as pd
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from dateutil import parser as dtparser

import tkinter as tk
from tkinter import ttk, filedialog, messagebox

from PIL import Image, ImageTk

try:
    import winsound
    _HAS_WINSOUND = True
except Exception:
    winsound = None
    _HAS_WINSOUND = False

try:
    import pygame
    _HAS_PYGAME = True
except Exception:
    pygame = None
    _HAS_PYGAME = False


@dataclass
class Annotation:
    video_path: str
    behavior: str
    kind: str
    start_frame: int
    end_frame: int
    start_time_s: float
    end_time_s: float
    start_datetime: str
    end_datetime: str


class BehaviourAnnotatorGUI:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("🦇 🎮 BAT : Behaviour Annotator Tool 🦇 🎮")
        self.root.geometry("1250x780")
        self.root.minsize(1100, 680)

        self.cap = None
        self.video_path = ""
        self.fps = 0.0
        self.frame_count = 0
        self.cur_frame_idx = 0
        self.playing = False
        self._last_rendered_idx = None
        self._tk_img = None

        self.start_dt = None

        self.behaviors = []
        self.annotations = []

        self.active_bouts = {}
        self.behavior_to_button = {}
        self.button_to_behavior = {}
        self.behavior_to_key = {}
        self.key_to_behavior = {}

        self._binding_behavior_btn = None
        self._binding_behavior_key = None

        self._pad = None
        self._pad_enabled = False
        self._pad_last_btn = {}
        self._pad_last_hat = (0, 0)
        self._pad_repeat_t = 0.0
        self._pad_id = None
        self._pad_name = ""
        self._pad_guid = ""

        self.sound_enabled = tk.BooleanVar(value=True)
        self.overlay_text = ""
        self.overlay_until = 0.0

        self._build_ui()

        self.root.bind("<KeyPress>", self._on_key_press)
        self.root.after(30, self._ui_tick)

        self._init_pygame()
        self._refresh_gamepads()

    def _build_ui(self):
        self.root.configure(bg="#111316")

        style = ttk.Style()
        try:
            style.theme_use("vista")
        except Exception:
            pass

        style.configure(".", background="#203C49", foreground="#0C4F69", fieldbackground="#2A2E33")
        style.configure("TFrame", background="#111316")
        style.map(
            "TButton",
            background=[("active", "#050505"), ("pressed", "#1A2F3C")],
            foreground=[("active", "#0D660D"), ("pressed", "#EC1717")],
        )
        style.configure("TLabel", background="#111316", foreground="#EAEAEA")
        style.configure("TButton", padding=6)
        style.configure("TLabelframe", background="#111316", foreground="#EAEAEA")
        style.configure("TLabelframe.Label", background="#111316", foreground="#EAEAEA")
        style.configure("TNotebook", background="#111316", borderwidth=0)
        style.configure("TNotebook.Tab", padding=(12, 8))
        style.map("TNotebook.Tab", background=[("selected", "#1A1F24")], foreground=[("selected", "#FFFFFF")])

        style.configure("Ethogram.TButton", padding=3)
        style.configure("Small.TButton", padding=4)

        top = ttk.Frame(self.root)
        top.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        btn_load = ttk.Button(top, text="Load video", command=self.load_video)
        btn_load.pack(side=tk.LEFT)

        self.video_label = ttk.Label(top, text="No video loaded")
        self.video_label.pack(side=tk.LEFT, padx=10)

        ttk.Label(top, text="FPS:").pack(side=tk.LEFT, padx=(20, 4))
        self.fps_var = tk.StringVar(value="")
        self.fps_entry = ttk.Entry(top, width=8, textvariable=self.fps_var)
        self.fps_entry.pack(side=tk.LEFT)
        ttk.Button(top, text="Apply FPS", command=self.apply_fps_override).pack(side=tk.LEFT, padx=6)

        ttk.Label(top, text="Start date/time (optional):").pack(side=tk.LEFT, padx=(20, 4))
        self.startdt_var = tk.StringVar(value="")
        self.startdt_entry = ttk.Entry(top, width=26, textvariable=self.startdt_var)
        self.startdt_entry.pack(side=tk.LEFT)
        ttk.Button(top, text="Set", command=self.apply_start_datetime).pack(side=tk.LEFT, padx=6)

        self.sound_chk = ttk.Checkbutton(top, text="Sounds", variable=self.sound_enabled)
        self.sound_chk.pack(side=tk.LEFT, padx=(20, 0))

        ttk.Button(top, text="Show controls", command=self.show_controls_popup).pack(side=tk.RIGHT)

        main = ttk.Frame(self.root)
        main.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        paned = ttk.Panedwindow(main, orient=tk.HORIZONTAL)
        paned.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        left = ttk.Frame(paned)
        right = ttk.Frame(paned, width=480)
        right.pack_propagate(False)

        paned.add(left, weight=4)
        paned.add(right, weight=1)

        vid_box = ttk.LabelFrame(left, text="Video")
        vid_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.canvas = tk.Canvas(vid_box, bg="#000000", highlightthickness=0)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        controls = ttk.Frame(vid_box)
        controls.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 8))

        self.btn_play = ttk.Button(controls, text="Play", command=self.toggle_play, state=tk.DISABLED)
        self.btn_play.pack(side=tk.LEFT)

        ttk.Button(controls, text="<< -100f", command=lambda: self.step_frames(-100), style="Small.TButton").pack(side=tk.LEFT, padx=6)
        ttk.Button(controls, text="< -1f", command=lambda: self.step_frames(-1), style="Small.TButton").pack(side=tk.LEFT)
        ttk.Button(controls, text="+1f >", command=lambda: self.step_frames(1), style="Small.TButton").pack(side=tk.LEFT, padx=6)
        ttk.Button(controls, text="+100f >>", command=lambda: self.step_frames(100), style="Small.TButton").pack(side=tk.LEFT)

        self.info_var = tk.StringVar(value="Frame: - / - | Time: - s | Timestamp: -")
        ttk.Label(controls, textvariable=self.info_var).pack(side=tk.RIGHT)

        self.slider = ttk.Scale(vid_box, from_=0, to=0, orient=tk.HORIZONTAL, command=self.on_slider)
        self.slider.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        help_row = ttk.Frame(vid_box)
        help_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        self.quick_help_var = tk.StringVar(value="")
        ttk.Label(help_row, textvariable=self.quick_help_var).pack(side=tk.LEFT, anchor="w")
        self._update_quick_help()

        nb = ttk.Notebook(right)
        nb.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        tab_eth = ttk.Frame(nb)
        tab_pad = ttk.Frame(nb)
        tab_ann = ttk.Frame(nb)

        nb.add(tab_eth, text="Ethogram")
        nb.add(tab_pad, text="Gamepad")
        nb.add(tab_ann, text="Annotations")

        eth_top = ttk.LabelFrame(tab_eth, text="Ethogram setup")
        eth_top.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        ttk.Label(eth_top, text="Behaviours (comma-separated):").pack(side=tk.TOP, anchor="w")
        self.eth_var = tk.StringVar(value="Replace with your behaviour")
        self.eth_entry = ttk.Entry(eth_top, textvariable=self.eth_var)
        self.eth_entry.pack(side=tk.TOP, fill=tk.X, pady=6)

        ttk.Button(eth_top, text="Apply ethogram", command=self.apply_ethogram).pack(side=tk.TOP, anchor="w")

        mode_row = ttk.Frame(eth_top)
        mode_row.pack(side=tk.TOP, fill=tk.X, pady=(10, 0))
        self.mode_var = tk.StringVar(value="bout")
        ttk.Radiobutton(mode_row, text="Bout mode (Start/End)", value="bout", variable=self.mode_var).pack(side=tk.LEFT)
        ttk.Radiobutton(mode_row, text="Event mode (Mark)", value="event", variable=self.mode_var).pack(side=tk.LEFT, padx=12)

        policy_row = ttk.Frame(eth_top)
        policy_row.pack(side=tk.TOP, fill=tk.X, pady=(6, 0))
        self.bout_policy_var = tk.StringVar(value="exclusive")
        ttk.Radiobutton(policy_row, text="Exclusive bouts", value="exclusive", variable=self.bout_policy_var).pack(side=tk.LEFT)
        ttk.Radiobutton(policy_row, text="Parallel bouts", value="parallel", variable=self.bout_policy_var).pack(side=tk.LEFT, padx=12)

        self.active_bout_var = tk.StringVar(value="Active bouts: none")
        ttk.Label(tab_eth, textvariable=self.active_bout_var).pack(side=tk.TOP, anchor="w", padx=12, pady=(0, 6))

        beh_box = ttk.LabelFrame(tab_eth, text="Ethogram buttons")
        beh_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        self.beh_scroll_canvas = tk.Canvas(beh_box, bg="#111316", highlightthickness=0)
        self.beh_scroll_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(10, 0), pady=10)

        beh_scrollbar = ttk.Scrollbar(beh_box, orient=tk.VERTICAL, command=self.beh_scroll_canvas.yview)
        beh_scrollbar.pack(side=tk.RIGHT, fill=tk.Y, padx=(0, 10), pady=10)

        self.beh_scroll_canvas.configure(yscrollcommand=beh_scrollbar.set)

        self.beh_btn_frame = ttk.Frame(self.beh_scroll_canvas)
        self.beh_btn_window = self.beh_scroll_canvas.create_window((0, 0), window=self.beh_btn_frame, anchor="nw")

        self.beh_btn_frame.bind("<Configure>", self._on_beh_frame_configure)
        self.beh_scroll_canvas.bind("<Configure>", self._on_beh_canvas_configure)
        self.beh_scroll_canvas.bind_all("<MouseWheel>", self._on_mousewheel)

        pad_box = ttk.LabelFrame(tab_pad, text="Gamepad")
        pad_box.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        self.pad_status_var = tk.StringVar(value="Gamepad: not initialised")
        ttk.Label(pad_box, textvariable=self.pad_status_var).pack(side=tk.TOP, anchor="w", padx=10, pady=(8, 6))

        pad_row = ttk.Frame(pad_box)
        pad_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 6))

        self.pad_list_var = tk.StringVar(value="")
        self.pad_combo = ttk.Combobox(pad_row, textvariable=self.pad_list_var, state="readonly", width=28, values=[])
        self.pad_combo.pack(side=tk.LEFT, fill=tk.X, expand=True)

        ttk.Button(pad_row, text="Refresh", command=self._refresh_gamepads).pack(side=tk.LEFT, padx=6)
        ttk.Button(pad_row, text="Connect", command=self._connect_selected_gamepad).pack(side=tk.LEFT)

        map_row = ttk.Frame(pad_box)
        map_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))
        ttk.Button(map_row, text="Save mapping", command=self.save_mapping).pack(side=tk.LEFT)
        ttk.Button(map_row, text="Load mapping", command=self.load_mapping).pack(side=tk.LEFT, padx=6)
        ttk.Button(map_row, text="Clear mapping", command=self.clear_mapping).pack(side=tk.LEFT, padx=6)
        ttk.Button(map_row, text="Show mapping", command=self.show_controls_popup).pack(side=tk.RIGHT)

        bind_box = ttk.LabelFrame(tab_pad, text="Bind behaviour keys/buttons")
        bind_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        self.bind_frame = ttk.Frame(bind_box)
        self.bind_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.bind_note_var = tk.StringVar(value="")
        ttk.Label(tab_pad, textvariable=self.bind_note_var).pack(side=tk.TOP, anchor="w", padx=12, pady=(0, 10))

        ann_box = ttk.LabelFrame(tab_ann, text="Annotations")
        ann_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        cols = ("behavior", "kind", "start_frame", "end_frame", "start_time_s", "end_time_s", "start_datetime", "end_datetime")
        self.tree = ttk.Treeview(ann_box, columns=cols, show="headings", height=12)
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=110 if c in ("behavior", "kind") else 120, anchor="center")
        self.tree.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        btn_row = ttk.Frame(ann_box)
        btn_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        ttk.Button(btn_row, text="Undo last", command=self.undo_last).pack(side=tk.LEFT)
        ttk.Button(btn_row, text="Delete selected", command=self.delete_selected).pack(side=tk.LEFT, padx=6)
        ttk.Button(btn_row, text="Clear all", command=self.clear_all).pack(side=tk.LEFT, padx=6)
        ttk.Button(btn_row, text="Save CSV", command=self.save_csv).pack(side=tk.RIGHT)

        footer = ttk.Frame(self.root)
        footer.pack(side=tk.BOTTOM, fill=tk.X)
        ttk.Label(
            footer,
            text="© 2026 BiRBSLAB | UiT | Norway",
            font=("Courier New", 10),
            foreground="orange",
        ).pack(pady=(2, 0))

        self.hyperlink_label = tk.Label(
            footer,
            text="Developed by Hamid Taghipourbibalan",
            font=("Courier New", 8),
            foreground="#1384e7",
            bg="#111316",
            cursor="hand2",
        )
        self.hyperlink_label.pack(pady=(0, 6))
        self.hyperlink_label.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

        self.apply_ethogram()

    def _set_overlay(self, text: str, duration_ms: int = 1200):
        self.overlay_text = text
        self.overlay_until = time.time() + max(0.05, duration_ms / 1000.0)
        if self.cap is not None:
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)

    def _play_sound(self, kind: str):
        if not bool(self.sound_enabled.get()):
            return
        if _HAS_WINSOUND:
            try:
                if kind == "start":
                    winsound.Beep(880, 90)
                    winsound.Beep(1320, 70)
                elif kind == "end":
                    winsound.Beep(1320, 70)
                    winsound.Beep(880, 90)
                elif kind == "event":
                    winsound.Beep(1046, 70)
                elif kind == "undo":
                    winsound.Beep(740, 80)
                elif kind == "warn":
                    winsound.Beep(330, 120)
                else:
                    winsound.Beep(880, 60)
            except Exception:
                try:
                    self.root.bell()
                except Exception:
                    pass
        else:
            try:
                self.root.bell()
            except Exception:
                pass

    def _on_mousewheel(self, e):
        try:
            if self.beh_scroll_canvas.winfo_ismapped():
                self.beh_scroll_canvas.yview_scroll(int(-1 * (e.delta / 120)), "units")
        except Exception:
            pass

    def _on_beh_frame_configure(self, _e=None):
        try:
            self.beh_scroll_canvas.configure(scrollregion=self.beh_scroll_canvas.bbox("all"))
        except Exception:
            pass

    def _on_beh_canvas_configure(self, e=None):
        try:
            self.beh_scroll_canvas.itemconfig(self.beh_btn_window, width=self.beh_scroll_canvas.winfo_width())
            self._layout_ethogram_buttons()
        except Exception:
            pass

    def _update_quick_help(self):
        self.quick_help_var.set(
            "Keyboard: Space/Enter=Play/Pause | ←/→=±1 frame | ↑/↓=±100 frames | Gamepad: D-pad=±1/±100 | Start=Play/Pause | Back=Undo"
        )

    def _init_pygame(self):
        if not _HAS_PYGAME:
            self.pad_status_var.set("Gamepad: pygame not installed")
            return
        try:
            pygame.init()
            pygame.joystick.init()
            self.pad_status_var.set("Gamepad: ready")
        except Exception:
            self.pad_status_var.set("Gamepad: init failed")

    def _refresh_gamepads(self):
        if not _HAS_PYGAME:
            self.pad_combo.configure(values=[])
            self.pad_list_var.set("")
            self.pad_status_var.set("Gamepad: pygame not installed")
            return
        try:
            pygame.joystick.quit()
            pygame.joystick.init()
            n = pygame.joystick.get_count()
            items = []
            for i in range(n):
                js = pygame.joystick.Joystick(i)
                js.init()
                name = js.get_name()
                guid = ""
                try:
                    guid = js.get_guid()
                except Exception:
                    guid = ""
                items.append(f"{i}: {name} {('[' + guid + ']') if guid else ''}".strip())
            self.pad_combo.configure(values=items)
            if items:
                self.pad_list_var.set(items[0])
            else:
                self.pad_list_var.set("")
            self.pad_status_var.set(f"Gamepad: {n} detected")
        except Exception:
            self.pad_combo.configure(values=[])
            self.pad_list_var.set("")
            self.pad_status_var.set("Gamepad: refresh failed")

    def _connect_selected_gamepad(self):
        if not _HAS_PYGAME:
            messagebox.showwarning("Gamepad", "pygame not installed. Install with: pip install pygame")
            return
        sel = self.pad_list_var.get().strip()
        if not sel:
            messagebox.showwarning("Gamepad", "No gamepad selected.")
            return
        try:
            idx = int(sel.split(":")[0].strip())
        except Exception:
            messagebox.showwarning("Gamepad", "Could not parse selection.")
            return

        try:
            js = pygame.joystick.Joystick(idx)
            js.init()
            self._pad = js
            self._pad_enabled = True
            self._pad_id = idx
            self._pad_name = js.get_name()
            try:
                self._pad_guid = js.get_guid()
            except Exception:
                self._pad_guid = ""
            self._pad_last_btn = {}
            self._pad_last_hat = (0, 0)
            self._pad_repeat_t = 0.0
            self.pad_status_var.set(f"Gamepad: connected to {self._pad_name}")
        except Exception:
            self._pad = None
            self._pad_enabled = False
            self.pad_status_var.set("Gamepad: connect failed")

    def _build_bind_rows(self):
        for w in self.bind_frame.winfo_children():
            w.destroy()

        header = ttk.Frame(self.bind_frame)
        header.pack(side=tk.TOP, fill=tk.X, pady=(0, 4))
        ttk.Label(header, text="Behaviour").pack(side=tk.LEFT)
        ttk.Label(header, text="Btn").pack(side=tk.LEFT, padx=(80, 0))
        ttk.Label(header, text="Key").pack(side=tk.LEFT, padx=(40, 0))

        for b in self.behaviors:
            row = ttk.Frame(self.bind_frame)
            row.pack(side=tk.TOP, fill=tk.X, pady=2)

            ttk.Label(row, text=b, width=16).pack(side=tk.LEFT)

            btn_txt = tk.StringVar(value=self._fmt_btn_mapping(b))
            key_txt = tk.StringVar(value=self._fmt_key_mapping(b))

            lab_btn = ttk.Label(row, textvariable=btn_txt, width=10)
            lab_btn.pack(side=tk.LEFT, padx=(10, 0))

            ttk.Button(row, text="Bind Btn", command=lambda bb=b: self.start_bind_button(bb), style="Small.TButton").pack(side=tk.LEFT, padx=6)
            ttk.Button(row, text="Clear Btn", command=lambda bb=b: self.clear_bind_button(bb), style="Small.TButton").pack(side=tk.LEFT, padx=2)

            lab_key = ttk.Label(row, textvariable=key_txt, width=12)
            lab_key.pack(side=tk.LEFT, padx=(10, 0))

            ttk.Button(row, text="Bind Key", command=lambda bb=b: self.start_bind_key(bb), style="Small.TButton").pack(side=tk.LEFT, padx=6)
            ttk.Button(row, text="Clear Key", command=lambda bb=b: self.clear_bind_key(bb), style="Small.TButton").pack(side=tk.LEFT, padx=2)

            row._btn_txt = btn_txt
            row._key_txt = key_txt

    def _refresh_bind_labels(self):
        for w in self.bind_frame.winfo_children():
            if hasattr(w, "_btn_txt"):
                b = w.winfo_children()[0].cget("text")
                w._btn_txt.set(self._fmt_btn_mapping(b))
                w._key_txt.set(self._fmt_key_mapping(b))

    def _fmt_btn_mapping(self, behavior: str) -> str:
        v = self.behavior_to_button.get(behavior, None)
        return "-" if v is None else str(v)

    def _fmt_key_mapping(self, behavior: str) -> str:
        v = self.behavior_to_key.get(behavior, "")
        return "-" if not v else v

    def start_bind_button(self, behavior: str):
        if not self._pad_enabled or self._pad is None:
            messagebox.showwarning("Gamepad", "Connect a gamepad first.")
            return
        self._binding_behavior_btn = behavior
        self._binding_behavior_key = None
        self.bind_note_var.set(f"Press a gamepad button to bind: {behavior}")

    def start_bind_key(self, behavior: str):
        self._binding_behavior_key = behavior
        self._binding_behavior_btn = None
        self.bind_note_var.set(f"Press a keyboard key to bind: {behavior}")

    def clear_bind_button(self, behavior: str):
        old_btn = self.behavior_to_button.pop(behavior, None)
        if old_btn is not None:
            self.button_to_behavior.pop(old_btn, None)
        self._refresh_bind_labels()

    def clear_bind_key(self, behavior: str):
        old_key = self.behavior_to_key.pop(behavior, "")
        if old_key:
            self.key_to_behavior.pop(old_key, None)
        self._refresh_bind_labels()

    def _assign_button(self, behavior: str, btn_idx: int):
        old_btn = self.behavior_to_button.get(behavior, None)
        if old_btn is not None:
            self.button_to_behavior.pop(old_btn, None)

        old_beh = self.button_to_behavior.get(btn_idx, None)
        if old_beh is not None:
            self.behavior_to_button.pop(old_beh, None)

        self.behavior_to_button[behavior] = btn_idx
        self.button_to_behavior[btn_idx] = behavior
        self._refresh_bind_labels()

    def _assign_key(self, behavior: str, key_sym: str):
        old_key = self.behavior_to_key.get(behavior, "")
        if old_key:
            self.key_to_behavior.pop(old_key, None)

        old_beh = self.key_to_behavior.get(key_sym, None)
        if old_beh is not None:
            self.behavior_to_key.pop(old_beh, None)

        self.behavior_to_key[behavior] = key_sym
        self.key_to_behavior[key_sym] = behavior
        self._refresh_bind_labels()

    def save_mapping(self):
        if not self.behaviors:
            messagebox.showwarning("Mapping", "Define behaviours first.")
            return
        out_path = filedialog.asksaveasfilename(
            title="Save mapping",
            defaultextension=".json",
            initialfile="gamepad_mapping.json",
            filetypes=[("JSON", "*.json")],
        )
        if not out_path:
            return
        data = {
            "pad_name": self._pad_name,
            "pad_guid": self._pad_guid,
            "behaviors": list(self.behaviors),
            "behavior_to_button": {k: int(v) for k, v in self.behavior_to_button.items()},
            "behavior_to_key": {k: str(v) for k, v in self.behavior_to_key.items()},
        }
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        messagebox.showinfo("Saved", f"Saved:\n{out_path}")

    def load_mapping(self):
        path = filedialog.askopenfilename(
            title="Load mapping",
            filetypes=[("JSON", "*.json")],
        )
        if not path:
            return
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception:
            messagebox.showerror("Mapping", "Could not read mapping JSON.")
            return

        b2b = data.get("behavior_to_button", {}) or {}
        b2k = data.get("behavior_to_key", {}) or {}

        self.behavior_to_button.clear()
        self.button_to_behavior.clear()
        self.behavior_to_key.clear()
        self.key_to_behavior.clear()

        for beh, btn in b2b.items():
            try:
                btn_i = int(btn)
            except Exception:
                continue
            self.behavior_to_button[str(beh)] = btn_i
            self.button_to_behavior[btn_i] = str(beh)

        for beh, key in b2k.items():
            key_s = str(key)
            if not key_s:
                continue
            self.behavior_to_key[str(beh)] = key_s
            self.key_to_behavior[key_s] = str(beh)

        self._refresh_bind_labels()
        self.bind_note_var.set("Mapping loaded")

    def clear_mapping(self):
        self.behavior_to_button.clear()
        self.button_to_behavior.clear()
        self.behavior_to_key.clear()
        self.key_to_behavior.clear()
        self._refresh_bind_labels()
        self.bind_note_var.set("Mapping cleared")

    def _on_key_press(self, e: tk.Event):
        if self._binding_behavior_key is not None:
            beh = self._binding_behavior_key
            self._binding_behavior_key = None
            self._binding_behavior_btn = None
            key_sym = e.keysym
            self._assign_key(beh, key_sym)
            self.bind_note_var.set(f"Bound key {key_sym} to {beh}")
            self._play_sound("event")
            return

        key_sym = e.keysym
        beh = self.key_to_behavior.get(key_sym, None)
        if beh is not None:
            self.on_behavior_click(beh)
            return

        if key_sym in ("space", "Return"):
            self.toggle_play()
        elif key_sym in ("Left",):
            self.step_frames(-1)
        elif key_sym in ("Right",):
            self.step_frames(1)
        elif key_sym in ("Up",):
            self.step_frames(-100)
        elif key_sym in ("Down",):
            self.step_frames(100)

    def load_video(self):
        path = filedialog.askopenfilename(
            title="Select video",
            filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv *.m4v"), ("All files", "*.*")],
        )
        if not path:
            return

        self._close_video()

        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            messagebox.showerror("Error", "Could not open video.")
            return

        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps is None or fps <= 0 or math.isnan(fps):
            fps = 0.0

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)

        self.cap = cap
        self.video_path = path
        self.fps = float(fps)
        self.frame_count = frame_count
        self.cur_frame_idx = 0
        self.playing = False
        self._last_rendered_idx = None

        self.video_label.config(text=os.path.basename(path))
        self.fps_var.set("" if self.fps <= 0 else f"{self.fps:.3f}")

        self.slider.configure(from_=0, to=max(0, self.frame_count - 1))
        self.slider.set(0)

        self.btn_play.config(state=tk.NORMAL, text="Play")

        self._set_overlay("VIDEO LOADED", 900)
        self._play_sound("event")
        self.render_frame(0)

    def _close_video(self):
        if self.cap is not None:
            try:
                self.cap.release()
            except Exception:
                pass
        self.cap = None

    def apply_fps_override(self):
        txt = self.fps_var.get().strip()
        if not txt:
            return
        try:
            fps = float(txt)
            if fps <= 0:
                raise ValueError
            self.fps = fps
            messagebox.showinfo("FPS set", f"FPS set to {self.fps:.3f}")
            self._update_info_label()
            self._set_overlay(f"FPS = {self.fps:.3f}", 900)
            self._play_sound("event")
        except Exception:
            messagebox.showerror("Invalid FPS", "Please enter a valid FPS number (e.g., 30 or 29.97).")
            self._play_sound("warn")

    def apply_start_datetime(self):
        txt = self.startdt_var.get().strip()
        if not txt:
            self.start_dt = None
            self._update_info_label()
            messagebox.showinfo("Start datetime cleared", "No absolute timestamps will be computed.")
            self._set_overlay("TIMESTAMP: OFF", 900)
            self._play_sound("event")
            return
        try:
            dt = dtparser.parse(txt)
            self.start_dt = dt
            self._update_info_label()
            messagebox.showinfo("Start datetime set", f"Start datetime set to:\n{dt.isoformat(sep=' ')}")
            self._set_overlay("TIMESTAMP: ON", 900)
            self._play_sound("event")
        except Exception:
            messagebox.showerror(
                "Invalid datetime",
                "Could not parse date/time.\nExamples:\n- 2026-02-05 21:15:00\n- 05/02/2026 21:15\n- 2026-02-05T21:15:00",
            )
            self._play_sound("warn")

    def apply_ethogram(self):
        raw = self.eth_var.get().strip()
        behaviors = [b.strip() for b in raw.split(",") if b.strip()]
        if not behaviors:
            behaviors = ["Behaviour1"]

        self.behaviors = behaviors

        for w in self.beh_btn_frame.winfo_children():
            w.destroy()

        for b in self.behaviors:
            btn = ttk.Button(self.beh_btn_frame, text=b, command=lambda bb=b: self.on_behavior_click(bb), style="Ethogram.TButton")
            btn._beh = b

        self._layout_ethogram_buttons()

        self._build_bind_rows()
        self._refresh_active_bout_label()
        self._refresh_bind_labels()

    def _layout_ethogram_buttons(self):
        for w in self.beh_btn_frame.winfo_children():
            w.grid_forget()

        btns = list(self.beh_btn_frame.winfo_children())
        if not btns:
            return

        width = max(220, self.beh_scroll_canvas.winfo_width())
        col_w = 200
        cols = max(1, min(4, width // col_w))

        for i in range(cols):
            self.beh_btn_frame.grid_columnconfigure(i, weight=1)

        for idx, btn in enumerate(btns):
            r = idx // cols
            c = idx % cols
            btn.grid(row=r, column=c, sticky="ew", padx=6, pady=4)

        self._on_beh_frame_configure()

    def toggle_play(self):
        if self.cap is None:
            return
        self.playing = not self.playing
        self.btn_play.config(text="Pause" if self.playing else "Play")
        self._set_overlay("PLAY" if self.playing else "PAUSE", 650)
        self._play_sound("event")

    def step_frames(self, n: int):
        if self.cap is None:
            return
        self.playing = False
        self.btn_play.config(text="Play")
        new_idx = max(0, min(self.frame_count - 1, self.cur_frame_idx + n))
        self.cur_frame_idx = new_idx
        self.slider.set(new_idx)
        self.render_frame(new_idx)

    def on_slider(self, _val):
        if self.cap is None:
            return
        idx = int(float(self.slider.get()))
        if idx != self.cur_frame_idx:
            self.playing = False
            self.btn_play.config(text="Play")
            self.cur_frame_idx = idx
            self.render_frame(idx)

    def _ui_tick(self):
        self._poll_gamepad()

        if self.cap is not None and self.playing:
            nxt = self.cur_frame_idx + 1
            if nxt >= self.frame_count:
                self.playing = False
                self.btn_play.config(text="Play")
                self._set_overlay("END OF VIDEO", 900)
                self._play_sound("end")
            else:
                self.cur_frame_idx = nxt
                self.slider.set(nxt)
                self.render_frame(nxt)

        self.root.after(30, self._ui_tick)

    def _pad_btn_pressed(self, btn_idx: int) -> bool:
        if not self._pad_enabled or self._pad is None:
            return False
        v = int(self._pad.get_button(btn_idx))
        prev = self._pad_last_btn.get(btn_idx, 0)
        self._pad_last_btn[btn_idx] = v
        return v == 1 and prev == 0

    def _poll_gamepad(self):
        if not self._pad_enabled or self._pad is None or not _HAS_PYGAME:
            return
        try:
            pygame.event.pump()
        except Exception:
            return

        if self._binding_behavior_btn is not None:
            beh = self._binding_behavior_btn
            try:
                nb = self._pad.get_numbuttons()
            except Exception:
                nb = 0
            for i in range(nb):
                if self._pad_btn_pressed(i):
                    self._binding_behavior_btn = None
                    self._binding_behavior_key = None
                    self._assign_button(beh, i)
                    self.bind_note_var.set(f"Bound button {i} to {beh}")
                    self._play_sound("event")
                    break
            return

        if self.cap is None:
            return

        try:
            if self._pad.get_numhats() > 0:
                hat = self._pad.get_hat(0)
            else:
                hat = (0, 0)
        except Exception:
            hat = (0, 0)

        if hat != self._pad_last_hat:
            self._pad_last_hat = hat
            hx, hy = hat
            if hx == -1:
                self.step_frames(-1)
            elif hx == 1:
                self.step_frames(1)
            if hy == 1:
                self.step_frames(-100)
            elif hy == -1:
                self.step_frames(100)

        now = time.time()
        try:
            ax0 = float(self._pad.get_axis(0)) if self._pad.get_numaxes() > 0 else 0.0
        except Exception:
            ax0 = 0.0

        if abs(ax0) > 0.75 and now - self._pad_repeat_t > 0.08:
            self._pad_repeat_t = now
            self.step_frames(1 if ax0 > 0 else -1)

        if self._pad_btn_pressed(7):
            self.toggle_play()
        if self._pad_btn_pressed(6):
            self.undo_last()

        for btn_idx, beh in list(self.button_to_behavior.items()):
            if self._pad_btn_pressed(btn_idx):
                self.on_behavior_click(beh)
                break

    def _draw_hud(self, frame_bgr):
        h, w = frame_bgr.shape[:2]
        scale = max(0.6, min(1.2, w / 1280.0))
        thick = max(1, int(2 * scale))
        pad = int(14 * scale)

        lines = []
        if self.active_bouts:
            active = ", ".join(list(self.active_bouts.keys())[:8])
            if len(self.active_bouts) > 8:
                active += f" +{len(self.active_bouts) - 8}"
            lines.append(f"ACTIVE: {active}")
        else:
            lines.append("ACTIVE: none")

        mode = self.mode_var.get()
        pol = self.bout_policy_var.get()
        lines.append(f"MODE: {mode}   POLICY: {pol}")

        y = pad + int(12 * scale)
        for i, s in enumerate(lines):
            cv2.putText(frame_bgr, s, (pad, y + i * int(26 * scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.7 * scale, (0, 0, 0), thick + 2, cv2.LINE_AA)
            cv2.putText(frame_bgr, s, (pad, y + i * int(26 * scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.7 * scale, (245, 245, 245), thick, cv2.LINE_AA)

        if self.overlay_text and time.time() <= self.overlay_until:
            msg = self.overlay_text
            msg_scale = 1.15 * scale
            (tw, th), _ = cv2.getTextSize(msg, cv2.FONT_HERSHEY_SIMPLEX, msg_scale, thick + 1)
            x = max(pad, (w - tw) // 2)
            y0 = max(pad + th + 5, int(h * 0.10))
            cv2.putText(frame_bgr, msg, (x, y0), cv2.FONT_HERSHEY_SIMPLEX, msg_scale, (0, 0, 0), thick + 4, cv2.LINE_AA)
            cv2.putText(frame_bgr, msg, (x, y0), cv2.FONT_HERSHEY_SIMPLEX, msg_scale, (80, 220, 120), thick + 1, cv2.LINE_AA)

        return frame_bgr

    def render_frame(self, idx: int):
        if self.cap is None:
            return
        if self._last_rendered_idx == idx:
            self._update_info_label()
            return

        self.cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = self.cap.read()
        if not ok or frame is None:
            self._update_info_label()
            return

        self._last_rendered_idx = idx

        frame = self._draw_hud(frame)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        canvas_w = max(1, self.canvas.winfo_width())
        canvas_h = max(1, self.canvas.winfo_height())

        h, w = frame_rgb.shape[:2]
        scale = min(canvas_w / w, canvas_h / h)
        new_w = max(1, int(w * scale))
        new_h = max(1, int(h * scale))

        img = Image.fromarray(frame_rgb).resize((new_w, new_h), Image.Resampling.LANCZOS)
        self._tk_img = ImageTk.PhotoImage(img)

        self.canvas.delete("all")
        x0 = (canvas_w - new_w) // 2
        y0 = (canvas_h - new_h) // 2
        self.canvas.create_image(x0, y0, anchor="nw", image=self._tk_img)

        self._update_info_label()

    def _update_info_label(self):
        if self.cap is None or self.frame_count <= 0:
            self.info_var.set("Frame: - / - | Time: - s | Timestamp: -")
            return

        t_s = self._frame_to_seconds(self.cur_frame_idx)
        ts = "-"
        if self.start_dt is not None and self.fps and self.fps > 0:
            ts_dt = self.start_dt + timedelta(seconds=t_s)
            ts = ts_dt.isoformat(sep=" ")

        self.info_var.set(
            f"Frame: {self.cur_frame_idx} / {self.frame_count - 1} | "
            f"Time: {t_s:.3f} s | Timestamp: {ts}"
        )

    def _frame_to_seconds(self, frame_idx: int) -> float:
        if self.fps and self.fps > 0:
            return frame_idx / self.fps
        return float(frame_idx)

    def _frame_to_datetime_str(self, frame_idx: int) -> str:
        if self.start_dt is None or not (self.fps and self.fps > 0):
            return ""
        dt = self.start_dt + timedelta(seconds=self._frame_to_seconds(frame_idx))
        return dt.isoformat(sep=" ")

    def on_behavior_click(self, behavior: str):
        if self.cap is None:
            messagebox.showwarning("No video", "Load a video first.")
            self._play_sound("warn")
            return

        mode = self.mode_var.get()
        if mode == "event":
            self._add_event(behavior)
            self._set_overlay(f"EVENT: {behavior}", 900)
            self._play_sound("event")
            return

        policy = self.bout_policy_var.get()
        if policy == "parallel":
            if behavior in self.active_bouts:
                sf = int(self.active_bouts.get(behavior, self.cur_frame_idx))
                dur = self._frame_to_seconds(self.cur_frame_idx) - self._frame_to_seconds(sf)
                self._end_bout_for_behavior(behavior, self.cur_frame_idx)
                self._set_overlay(f"END: {behavior}  ({dur:.2f}s)", 1100)
                self._play_sound("end")
            else:
                self.active_bouts[behavior] = int(self.cur_frame_idx)
                self._set_overlay(f"START: {behavior}", 900)
                self._play_sound("start")
        else:
            if behavior in self.active_bouts:
                sf = int(self.active_bouts.get(behavior, self.cur_frame_idx))
                dur = self._frame_to_seconds(self.cur_frame_idx) - self._frame_to_seconds(sf)
                self._end_bout_for_behavior(behavior, self.cur_frame_idx)
                self._set_overlay(f"END: {behavior}  ({dur:.2f}s)", 1100)
                self._play_sound("end")
            else:
                ended_any = False
                for b in list(self.active_bouts.keys()):
                    sf = int(self.active_bouts.get(b, self.cur_frame_idx))
                    dur = self._frame_to_seconds(self.cur_frame_idx) - self._frame_to_seconds(sf)
                    self._end_bout_for_behavior(b, self.cur_frame_idx)
                    ended_any = True
                self.active_bouts[behavior] = int(self.cur_frame_idx)
                self._set_overlay(f"START: {behavior}" + ("  (auto-ended others)" if ended_any else ""), 1100)
                self._play_sound("start")

        self._refresh_active_bout_label()
        self._last_rendered_idx = None
        self.render_frame(self.cur_frame_idx)

    def _add_event(self, behavior: str):
        sf = self.cur_frame_idx
        ef = self.cur_frame_idx
        st = self._frame_to_seconds(sf)
        et = self._frame_to_seconds(ef)
        sd = self._frame_to_datetime_str(sf)
        ed = self._frame_to_datetime_str(ef)

        ann = Annotation(
            video_path=self.video_path,
            behavior=behavior,
            kind="event",
            start_frame=sf,
            end_frame=ef,
            start_time_s=st,
            end_time_s=et,
            start_datetime=sd,
            end_datetime=ed,
        )
        self.annotations.append(ann)
        self._tree_add(ann)

    def _end_bout_for_behavior(self, behavior: str, end_frame: int):
        if behavior not in self.active_bouts:
            return

        sf = int(self.active_bouts.get(behavior, end_frame))
        ef = int(end_frame)
        if ef < sf:
            ef = sf

        st = self._frame_to_seconds(sf)
        et = self._frame_to_seconds(ef)
        sd = self._frame_to_datetime_str(sf)
        ed = self._frame_to_datetime_str(ef)

        ann = Annotation(
            video_path=self.video_path,
            behavior=behavior,
            kind="bout",
            start_frame=sf,
            end_frame=ef,
            start_time_s=st,
            end_time_s=et,
            start_datetime=sd,
            end_datetime=ed,
        )

        self.active_bouts.pop(behavior, None)
        self.annotations.append(ann)
        self._tree_add(ann)

    def _refresh_active_bout_label(self):
        if not self.active_bouts:
            self.active_bout_var.set("Active bouts: none")
        else:
            parts = []
            for b, sf in self.active_bouts.items():
                parts.append(f"{b}@{sf}")
            self.active_bout_var.set("Active bouts: " + ", ".join(parts))

    def _tree_add(self, ann: Annotation):
        vals = (
            ann.behavior,
            ann.kind,
            ann.start_frame,
            ann.end_frame,
            f"{ann.start_time_s:.6f}",
            f"{ann.end_time_s:.6f}",
            ann.start_datetime,
            ann.end_datetime,
        )
        self.tree.insert("", tk.END, values=vals)

    def undo_last(self):
        if not self.annotations:
            return
        self.annotations.pop()
        children = self.tree.get_children()
        if children:
            self.tree.delete(children[-1])
        self._set_overlay("UNDO", 700)
        self._play_sound("undo")
        if self.cap is not None:
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)

    def delete_selected(self):
        sel = self.tree.selection()
        if not sel:
            return
        for item in sel:
            self.tree.delete(item)
        self._rebuild_annotations_from_tree()
        self._set_overlay("DELETED", 700)
        self._play_sound("undo")
        if self.cap is not None:
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)

    def clear_all(self):
        if not self.annotations and not self.tree.get_children():
            return
        if messagebox.askyesno("Clear all", "Delete ALL saved annotations?"):
            self.annotations = []
            for item in self.tree.get_children():
                self.tree.delete(item)
            self._set_overlay("CLEARED", 700)
            self._play_sound("undo")
            if self.cap is not None:
                self._last_rendered_idx = None
                self.render_frame(self.cur_frame_idx)

    def _rebuild_annotations_from_tree(self):
        anns = []
        for item in self.tree.get_children():
            v = self.tree.item(item, "values")
            behavior = v[0]
            kind = v[1]
            sf = int(v[2])
            ef = int(v[3])
            st = float(v[4])
            et = float(v[5])
            sd = v[6]
            ed = v[7]
            anns.append(
                Annotation(
                    video_path=self.video_path,
                    behavior=behavior,
                    kind=kind,
                    start_frame=sf,
                    end_frame=ef,
                    start_time_s=st,
                    end_time_s=et,
                    start_datetime=sd,
                    end_datetime=ed,
                )
            )
        self.annotations = anns

    def save_csv(self):
        if not self.annotations and not self.active_bouts:
            messagebox.showwarning("Nothing to save", "No annotations yet.")
            self._play_sound("warn")
            return

        if self.active_bouts:
            if messagebox.askyesno("Active bouts", "You have active bouts. End ALL at the current frame before saving?"):
                for b in list(self.active_bouts.keys()):
                    sf = int(self.active_bouts.get(b, self.cur_frame_idx))
                    dur = self._frame_to_seconds(self.cur_frame_idx) - self._frame_to_seconds(sf)
                    self._end_bout_for_behavior(b, self.cur_frame_idx)
                self._refresh_active_bout_label()
                self._set_overlay("ENDED ALL ACTIVE", 900)
                self._play_sound("end")

        if not self.annotations:
            messagebox.showwarning("Nothing to save", "No annotations to save.")
            self._play_sound("warn")
            return

        default_name = "annotations.csv"
        if self.video_path:
            base = os.path.splitext(os.path.basename(self.video_path))[0]
            default_name = f"{base}_annotations.csv"

        out_path = filedialog.asksaveasfilename(
            title="Save CSV",
            defaultextension=".csv",
            initialfile=default_name,
            filetypes=[("CSV", "*.csv")],
        )
        if not out_path:
            return

        rows = [asdict(a) for a in self.annotations]

        for r in rows:
            r["fps_used"] = self.fps
            r["frame_count"] = self.frame_count
            r["video_filename"] = os.path.basename(self.video_path)
            r["start_datetime_input"] = "" if self.start_dt is None else self.start_dt.isoformat(sep=" ")
            r["mode"] = self.mode_var.get()
            r["bout_policy"] = self.bout_policy_var.get()
            r["pad_name"] = self._pad_name
            r["pad_guid"] = self._pad_guid

        df = pd.DataFrame(rows)
        df.to_csv(out_path, index=False)

        messagebox.showinfo("Saved", f"Saved:\n{out_path}")
        self._set_overlay("SAVED CSV", 900)
        self._play_sound("event")

    def show_controls_popup(self):
        win = tk.Toplevel(self.root)
        win.title("Controls / Mapping")
        win.geometry("780x520")
        win.configure(bg="#111316")

        box = ttk.Frame(win)
        box.pack(fill=tk.BOTH, expand=True, padx=12, pady=12)

        txt = tk.Text(box, wrap="word", bg="#0E1013", fg="#EAEAEA", insertbackground="#EAEAEA")
        txt.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        sb = ttk.Scrollbar(box, orient="vertical", command=txt.yview)
        sb.pack(side=tk.RIGHT, fill=tk.Y)
        txt.configure(yscrollcommand=sb.set)

        lines = []
        lines.append("VIDEO CONTROLS")
        lines.append("  Keyboard: Space/Enter = Play/Pause")
        lines.append("            ← / →      = -1 / +1 frame")
        lines.append("            ↑ / ↓      = -100 / +100 frames")
        lines.append("")
        lines.append("  Gamepad (default indices; may differ by controller):")
        lines.append("            D-pad left/right = -1 / +1 frame")
        lines.append("            D-pad up/down    = -100 / +100 frames")
        lines.append("            Left stick X     = continuous seek")
        lines.append("            Button 7 (Start) = Play/Pause")
        lines.append("            Button 6 (Back)  = Undo last annotation")
        lines.append("")
        lines.append("ANNOTATION BEHAVIOURS")
        if not self.behaviors:
            lines.append("  (No behaviours defined)")
        else:
            for b in self.behaviors:
                kb = self.behavior_to_key.get(b, "-") or "-"
                gb = self.behavior_to_button.get(b, None)
                gb_s = "-" if gb is None else str(gb)
                lines.append(f"  {b}:  key={kb}   btn={gb_s}")
        lines.append("")
        lines.append("BOUT MODE LOGIC")
        lines.append("  Bout mode: press a behaviour once to START, press same behaviour again to END.")
        lines.append("  Exclusive bouts: starting a new behaviour ends any currently active bout(s).")
        lines.append("  Parallel bouts: each behaviour toggles independently.")
        lines.append("")
        lines.append("HUD / FEEDBACK")
        lines.append("  Video overlay shows currently ACTIVE bouts at top-left.")
        lines.append("  START/END/EVENT flashes briefly in the video.")
        lines.append("  Toggle Sounds from the top bar.")

        txt.insert("1.0", "\n".join(lines))
        txt.configure(state="disabled")

    def open_hyperlink(self, url: str):
        import webbrowser
        webbrowser.open(url)


def main():
    root = tk.Tk()
    app = BehaviourAnnotatorGUI(root)
    root.mainloop()


if __name__ == "__main__":
    main()


c:\Users\hta031\AppData\Local\miniconda3\envs\ACCBIRDS\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html
